In [0]:
import pandas as pd
import numpy as np

In [2]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


     |████████████████████████████████| 993kB 5.0MB/s 


In [0]:
# List .csv files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '.csv'"}).GetList()

# The code below will output a list of all the .csv files
# for file in listed:
#   print('title {}, id {}'.format(file['title'], file['id']))

In [0]:
# Download the "modes performance to visualize.csv"

file_id = '0Bw7LkoymU5okWC10Yk1sTThkdDBxRGo3X1ZoZWVyVGhYcWgw'
downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
downloaded.GetContentFile("performance.csv")

In [5]:
performance_df = pd.read_csv('performance.csv')
print(performance_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 27 columns):
variables                   5 non-null object
RMSE                        5 non-null float64
SD.Error                    5 non-null float64
RMSE.Actual.change          5 non-null float64
SD.Actual.change            5 non-null float64
MPE                         5 non-null float64
SD.PE                       5 non-null float64
MAPE                        5 non-null float64
SD.APE                      5 non-null float64
Accuracy.2X2                5 non-null float64
F1.score.Up                 5 non-null float64
F1.score.Down               5 non-null float64
Accuracy.3X3.5.             5 non-null float64
Bad.predictions.3X3.5.      5 non-null float64
Accuracy.3X3.2.5.           5 non-null float64
Bad.predictions.3X3.2.5.    5 non-null float64
F1_ave                      5 non-null float64
F1_ave7525                  5 non-null float64
RMSE_point                  5 non-null int64
F1up_poin

In [6]:
performance_df.head()

,variables,RMSE,SD.Error,RMSE.Actual.change,SD.Actual.change,MPE,SD.PE,MAPE,SD.APE,Accuracy.2X2,F1.score.Up,F1.score.Down,Accuracy.3X3.5.,Bad.predictions.3X3.5.,Accuracy.3X3.2.5.,Bad.predictions.3X3.2.5.,F1_ave,F1_ave7525,RMSE_point,F1up_point,F1down_point,F1ave_point,F1ave7525_point,A3x3_point,B3x3_point,points_total,Top_models
0,"VECM_1_1,P1A_03,AIS_Panamax,TC5~TCE,P2A_03,BCI...",3491.099371,3483.280991,3338.171179,3336.861478,-4.469601,51.583631,31.758544,40.650574,0.518192,0.525777,0.510091,0.372165,0.256649,0.423071,0.358786,0.517934,0.521855,2,0,2,1,0,1,1,7,1
1,"VECM_1_2,P1A_03,AIS_Panamax,ICE_TIB4,ICE_SB1,I...",3460.338476,3457.980938,3338.171179,3336.861478,-6.867125,55.897946,33.109500,45.449799,0.524392,0.534923,0.513188,0.373470,0.281612,0.430250,0.372981,0.524055,0.529489,2,2,2,1,1,1,0,8,1
2,"VAR,P1A_03,AIS_Panamax,ICE_NCF2,CME_CL2,ICE_TI...",3124.424950,3096.653697,3338.171179,3336.861478,-11.207729,50.752213,30.133463,42.310283,0.624245,0.661908,0.572128,0.493066,0.190243,0.542013,0.267580,0.617018,0.639463,2,2,2,1,1,1,1,9,1
3,"VAR,P1A_03,AIS_Panamax,ICE_NCF2,ICE_ATW2,CME_N...",3069.582807,3062.778685,3338.171179,3336.861478,-9.773460,48.576517,29.103193,40.050002,0.644967,0.667972,0.616023,0.514929,0.187143,0.567629,0.261707,0.641997,0.654984,2,2,2,1,1,1,1,9,1
4,"VAR,P1A_03,AIS_Panamax,CME_NG1,ICE_TIB2,ICE_AT...",3069.582807,3062.778685,3338.171179,3336.861478,-9.773460,48.576517,29.103193,40.050002,0.644967,0.667972,0.616023,0.514929,0.187143,0.567629,0.261707,0.641997,0.654984,2,2,2,1,1,1,1,9,1


Visualizations

In [0]:
feature_lst = []
for i in range(performance_df.shape[0]):
  feature_lst+= performance_df.variables.str.split(",")[i]

In [8]:
s= feature_lst
feature_df = performance_df.variables.str.get_dummies(sep=',').reindex(columns=s)
feature_df.shape

(5, 80)

In [9]:
_,i = np.unique(feature_df.columns, return_index=True)
feature_df = feature_df.iloc[:, i]
feature_df.shape

(5, 29)

In [16]:
feature_df.T

,0,1,2,3,4
AIS_Panamax,1,1,1,1,1
AWS,0,0,1,1,1
BCI,1,0,0,0,0
CME_CL2,0,0,1,0,0
CME_NG1,0,0,1,1,1
DB_1,1,1,0,0,0
DB_3,1,1,0,0,0
DB_4,0,0,1,1,1
DB_5,1,1,1,1,1
DB_6,0,0,1,1,1


In [11]:
performance_df.merge(feature_df, on=feature_df.index)

,key_0,variables,RMSE,SD.Error,RMSE.Actual.change,SD.Actual.change,MPE,SD.PE,MAPE,SD.APE,Accuracy.2X2,F1.score.Up,F1.score.Down,Accuracy.3X3.5.,Bad.predictions.3X3.5.,Accuracy.3X3.2.5.,Bad.predictions.3X3.2.5.,F1_ave,F1_ave7525,RMSE_point,F1up_point,F1down_point,F1ave_point,F1ave7525_point,A3x3_point,B3x3_point,points_total,Top_models,AIS_Panamax,AWS,BCI,CME_CL2,CME_NG1,DB_1,DB_3,DB_4,DB_5,DB_6,DL_2,DL_3,DP_2,DP_3,DP_6,ICE_ATW2,ICE_NCF2,ICE_SB1,ICE_TIB2,ICE_TIB4,P1A_03,P2A_03,RICI,SB,SL,TC5~TCE,VAR,VECM_1_1,VECM_1_2
0,0,"VECM_1_1,P1A_03,AIS_Panamax,TC5~TCE,P2A_03,BCI...",3491.099371,3483.280991,3338.171179,3336.861478,-4.469601,51.583631,31.758544,40.650574,0.518192,0.525777,0.510091,0.372165,0.256649,0.423071,0.358786,0.517934,0.521855,2,0,2,1,0,1,1,7,1,1,0,1,0,0,1,1,0,1,0,1,1,1,0,1,0,0,0,0,0,1,1,1,1,1,1,0,1,0
1,1,"VECM_1_2,P1A_03,AIS_Panamax,ICE_TIB4,ICE_SB1,I...",3460.338476,3457.980938,3338.171179,3336.861478,-6.867125,55.897946,33.109500,45.449799,0.524392,0.534923,0.513188,0.373470,0.281612,0.430250,0.372981,0.524055,0.529489,2,2,2,1,1,1,0,8,1,1,0,0,0,0,1,1,0,1,0,1,1,1,0,1,1,0,1,0,1,1,0,1,1,1,0,0,0,1
2,2,"VAR,P1A_03,AIS_Panamax,ICE_NCF2,CME_CL2,ICE_TI...",3124.424950,3096.653697,3338.171179,3336.861478,-11.207729,50.752213,30.133463,42.310283,0.624245,0.661908,0.572128,0.493066,0.190243,0.542013,0.267580,0.617018,0.639463,2,2,2,1,1,1,1,9,1,1,1,0,1,1,0,0,1,1,1,1,0,1,1,0,0,1,0,1,0,1,0,0,1,1,0,1,0,0
3,3,"VAR,P1A_03,AIS_Panamax,ICE_NCF2,ICE_ATW2,CME_N...",3069.582807,3062.778685,3338.171179,3336.861478,-9.773460,48.576517,29.103193,40.050002,0.644967,0.667972,0.616023,0.514929,0.187143,0.567629,0.261707,0.641997,0.654984,2,2,2,1,1,1,1,9,1,1,1,0,0,1,0,0,1,1,1,1,0,1,1,0,1,1,0,1,0,1,0,0,1,1,0,1,0,0
4,4,"VAR,P1A_03,AIS_Panamax,CME_NG1,ICE_TIB2,ICE_AT...",3069.582807,3062.778685,3338.171179,3336.861478,-9.773460,48.576517,29.103193,40.050002,0.644967,0.667972,0.616023,0.514929,0.187143,0.567629,0.261707,0.641997,0.654984,2,2,2,1,1,1,1,9,1,1,1,0,0,1,0,0,1,1,1,1,0,1,1,0,1,1,0,1,0,1,0,0,1,1,0,1,0,0


In [0]:
import plotly
plotly.__version__
plotly.tools.set_credentials_file(username='ritalulu123', api_key='mcPsXGYiHGp6rrOlr7YQ')

In [0]:
import plotly.plotly as py
import plotly.graph_objs as go


Visualize RMSE

In [19]:
data = [
    go.Bar(
        x=performance_df['variables'], # assign x as the dataframe column 'x'
        y=performance_df['RMSE']
    )
]

# IPython notebook
py.iplot(data, filename='pandas-bar-chart')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~ritalulu123/0 or inside your plot.ly account where it is named 'pandas-bar-chart'


Radar chart

In [0]:
data = [{
  type: 'scatterpolar',
  r: [39, 28, 8, 7, 28, 39],
  theta: ['A','B','C', 'D', 'E', 'A'],
  fill: 'toself'
}]

layout = {
  polar: {
    radialaxis: {
      visible: true,
      range: [0, 50]
    }
  },
  showlegend: false
}

Plotly.plot("myDiv", data, layout)